<a href="https://colab.research.google.com/github/JuanCruzArese/TP-Final-ICD/blob/laburen-aca/TP_Final_ICD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/JuanCruzArese/TP-Final-ICD
%cd TP-Final-ICD

## Importar librerias

In [ ]:
import pandas as pd # Para manejo de DataFrames
import numpy as np
import matplotlib.pyplot as plt # Para control de figuras
import seaborn as sns # Para visualizaciones (boxplot, heatmap)

from sklearn.model_selection import train_test_split, GridSearchCV # Para dividir datos en train/test y busqueda de grilla.
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler # Para escalado robusto
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression # Modelo Regresión Logística
from sklearn.ensemble import RandomForestClassifier # Modelo Random Forest
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (                          # Métricas de evaluación
    accuracy_score, precision_score, recall_score,
    roc_auc_score, confusion_matrix, f1_score, classification_report, ConfusionMatrixDisplay,
                                                       roc_curve, auc
)
from sklearn.feature_selection import SelectKBest, f_classif # Selector y criterio ANOVA F, respectivamente.

## Importar dataset

In [ ]:
df = pd.read_csv('/content/TP-Final-ICD/UCI_Credit_Card_TF.csv')

# **EXPLORACION Y ANALISIS DEL DATASET**

### Vistazo al dataframe

In [ ]:
df.head()

### Informacion de las columnas del dataframe (tipo de dato, entradas no nulas, etc.)

In [ ]:
df.info()

### Informacion estadistica del dataframe

In [ ]:
df.describe()

### Dimensiones del dataframe

In [ ]:
print(df.shape)

### Cantidad de clientes de tarjetas de credito

In [ ]:
print(len(df))

In [ ]:
def get_percentage(default_payment):
  return len(df[df['default.payment.next.month'] == default_payment]) * 100 / len(df)

### Porcentaje de clientes que pagaron

In [ ]:
print(get_percentage(0))

### Porcentaje de clientes que no pagaron

In [ ]:
print(get_percentage(1))

## Visualizacion

In [ ]:
# for column in df.columns:
#   plt.hist(df[column])
#   plt.title(column)
#   plt.show()

def plot(plot, x, y, title, ylabel):
  plt.figure(figsize=(12, 8))

  match plot:
    case 'box':
      sns.boxplot(data=df, x=x, y=y)
    case 'violin':
      sns.violinplot(data=df, x=x, y=y, inner='quartile')

  plt.title(title)
  plt.xlabel('')
  plt.xticks([0, 1], ['cumplirá el pago', 'no cumplirá el pago'])
  plt.ylabel(ylabel)

plot('box', 'default.payment.next.month', 'LIMIT_BAL',
     '[ DIAGRAMA DE CAJA ] Etiqueta segun el monto del credito',
     'Credito (en millones de TWD)')

plot('violin', 'default.payment.next.month', 'AGE',
     '[ GRAFICO DE VIOLIN ] Etiqueta segun la edad',
     'EDAD')

plt.show()

#**LIMPIEZA Y PREPARACION DE LOS DATOS**

In [ ]:
# Eliminar valores nulos
df = df.dropna()


# VER SI SE PUEDE REEMPLAZAR POR LA MEDIA!!!!!!!!!!!!!!!!!!!!!!!!!!!



# separar columnas
X = df.drop('default.payment.next.month', axis=1)
y = df['default.payment.next.month']